In [1]:
import coremltools
import keras
from keras.models import Model
import tensorflow

In [2]:
from keras.callbacks import EarlyStopping
from seq2seq_util import Seq2SeqBatchGenerator
import numpy as np
from keras.layers import Dense, Input, LSTM, Masking
from keras.models import Model

In [3]:
in_vocab_size = 101
out_vocab_size = 87
latent_dim = 256

coreml_enc_in = Input(shape=(None, in_vocab_size), name="encoder_in")
coreml_enc_lstm = LSTM(latent_dim, return_state=True, name="encoder_lstm")
coreml_enc_out, _, _ = coreml_enc_lstm(coreml_enc_in)

coreml_encoder_model = Model(coreml_enc_in, coreml_enc_out)
coreml_encoder_model.output_layers = coreml_encoder_model._output_layers

coreml_encoder_model.load_weights("Es2EnCharEncoderWeights.h5")

In [4]:
import coremltools
from keras.layers.normalization.layer_normalization import *
from keras.layers.normalization.batch_normalization import *

coreml_encoder = coremltools.converters.convert(
    coreml_encoder_model,
    input_names="encodedSeq",
    output_names="ignored")

coreml_encoder.save("Es2EnCharEncoder.mlmodel")

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 66/66 [00:00<00:00, 1463.84 ops/s]


In [5]:
coreml_dec_in = Input(shape=(None, out_vocab_size))

coreml_dec_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
coreml_dec_lstm_out, _, _ = coreml_dec_lstm(coreml_dec_in)
coreml_dec_dense = Dense(out_vocab_size, activation="softmax")
coreml_dec_out = coreml_dec_dense(coreml_dec_lstm_out)

coreml_decoder_model = Model(coreml_dec_in, coreml_dec_out)
coreml_decoder_model.output_layers = coreml_decoder_model._output_layers

coreml_decoder_model.load_weights("Es2EnCharDecoderWeights.h5")

In [6]:
coreml_decoder = coremltools.converters.convert(
    coreml_decoder_model,
    input_names="encodedChar",
    output_names="nextCharProbs")

coreml_decoder.save("Es2EnCharDecoder.mlmodel")

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 80/80 [00:00<00:00, 1853.27 ops/s]


In [7]:
from coremltools.models.neural_network import quantization_utils

def convert_to_fp16(mlmodel_filename):
    model_fp32 = coremltools.models.MLModel(mlmodel_filename)
    spec_16bit = quantization_utils.quantize_weights(model_fp32, nbits=16)
    coremltools.utils.save(spec_16bit, f"{mlmodel_filename}16Bit.mlmodel")

下面这个单元格我也不知道为什么又没有`save`这个了，不过这步~~应该~~也只是为了减小体积，我就跳过好了

In [8]:
convert_to_fp16("Es2EnCharEncoder.mlmodel")
convert_to_fp16("Es2EnCharDecoder.mlmodel")

Quantizing using linear quantization
Quantizing layer tf_make_list_0_condition_re_initialize
Quantizing layer tf_make_list_0_condition
Quantizing layer model/encoder_lstm/PartitionedCall/while_renamed
Quantizing layer model/encoder_lstm/PartitionedCall/while/while_body_1152/while/MatMul_1
Quantizing layer model/encoder_lstm/PartitionedCall/while/while_body_1152/while/MatMul
Quantizing layer model/encoder_lstm/PartitionedCall/while_1_condition_re_initialize
Quantizing layer model/encoder_lstm/PartitionedCall/while_1_condition


AttributeError: module 'coremltools.models.utils' has no attribute 'save'

下面单元格移步原notebook执行

In [9]:
import json

with open("esCharToInt.json", "w") as f:
    json.dump(in_token2int, f)
with open("intToEnChar.json", "w") as f:
    json.dump(out_int2token, f)

NameError: name 'in_token2int' is not defined